In [ ]:
#!/usr/bin/env python
"""
STAGE 07 · RISE-PROBABILITY EXPLORATION & LEADERBOARDS
───────────────────────────────────────────────────────
For every prediction flavour written in Stage 06 it writes:

    Stage7_Rank_<flavour>_<YEAR>.csv
    Stage7_HistGrid_<flavour>_<YEAR>.png
    Stage7_Boxplot_<flavour>_<YEAR>.png

If both *speed* and *depth* exist it also writes

    Stage7_Speed_vs_Depth_<YEAR>.csv
    Stage7_SpeedDepth_DiffBar_<YEAR>.png
"""
from __future__ import annotations
from pathlib import Path
from typing import Dict, List
import math, logging, os, warnings

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from pipeline_utils import load_cfg, resolve_run_dir

# ───────────────────────── CONFIG / PATHS ─────────────────────────
CFG        = load_cfg()
EVENTS     = {str(k): v for k, v in CFG["events"].items()}

SWAN_YEAR  = str(os.getenv("SWAN_YEAR") or next(iter(EVENTS)))
RUN_DIR    = resolve_run_dir(
                swan_year = SWAN_YEAR,
                must_have = f"stage06/Stage6Speed_RISE_Predictions_{SWAN_YEAR}.csv",
                run_tag   = os.getenv("RUN_TAG"))
STAGE_DIR  = RUN_DIR / "stage07"; STAGE_DIR.mkdir(exist_ok=True)

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(levelname)-7s | %(message)s",
                    handlers=[logging.FileHandler(STAGE_DIR/"stage07.log","w","utf-8"),
                              logging.StreamHandler()])
log = logging.getLogger(__name__)
warnings.filterwarnings("ignore", category=RuntimeWarning)
plt.rcParams["figure.dpi"] = 110
sns.set_style("whitegrid")

# ─────────────────────  prediction flavours  ──────────────────────
FLAV_INFO: Dict[str, Dict] = {
    "speed": {"file": RUN_DIR/f"stage06/Stage6Speed_RISE_Predictions_{SWAN_YEAR}.csv",
              "suffix": "_RISE_prob",
              "title":  "Speed model"},
    "depth": {"file": RUN_DIR/f"stage06/Stage6Depth_RISE_Predictions_{SWAN_YEAR}.csv",
              "suffix": "_DepthRISE_prob",
              "title":  "Depth model"},
    "blend": {"file": RUN_DIR/f"stage06/Stage6Blend_RISE_Predictions_{SWAN_YEAR}.csv",
              "suffix": "_BlendRISE_prob",
              "title":  "Blend (speed + depth)"},
}

# ────────────────────────── HELPERS ───────────────────────────────
def make_leaderboard(df: pd.DataFrame, probs: List[str], suf: str) -> pd.DataFrame:
    lb = (df[probs].mean()
            .sort_values(ascending=False)
            .rename("MeanProb")
            .reset_index()
            .rename(columns={"index": "Metric"}))
    lb["Metric"] = lb["Metric"].str.replace(suf, "", regex=False)
    return lb

def hist_grid(df: pd.DataFrame, cols: List[str], title: str, out: Path) -> None:
    n, c = len(cols), 4
    r    = -(-n // c)
    fig, axs = plt.subplots(r, c, figsize=(4*c, 3*r))
    for ax, col in zip(axs.flatten(), cols):
        df[col].dropna().hist(bins=20, edgecolor="k", ax=ax)
        ax.set_xlim(0, 1)
        ax.set_title(col.replace("_".join(col.split("_")[-3:]), ""))  # tidy metric name
    for ax in axs.flatten()[n:]:
        ax.axis("off")
    plt.suptitle(title, y=1.02)
    plt.tight_layout()
    plt.savefig(out, dpi=110)
    plt.close()

def box_plot(df: pd.DataFrame, cols: List[str], title: str, out: Path) -> None:
    plt.figure(figsize=(max(6, .6*len(cols)), 5))
    df[cols].boxplot(rot=45)
    plt.title(title)
    plt.ylabel("Predicted probability")
    plt.tight_layout()
    plt.savefig(out, dpi=110)
    plt.close()

# ────────────────────────── MAIN LOOP ─────────────────────────────
log.info("==========  STAGE 07 (%s) ==========", SWAN_YEAR)

flavour_lb: Dict[str, pd.DataFrame] = {}

for flav, info in FLAV_INFO.items():
    fp = info["file"]
    if not fp.exists() or fp.stat().st_size == 0:
        log.warning("%s missing – %s flavour skipped", fp.name, flav)
        continue

    df  = pd.read_csv(fp)
    suf = info["suffix"]
    pcols = [c for c in df.columns if c.endswith(suf)]
    if not pcols:
        log.warning("No probability columns with suffix %s in %s", suf, fp.name)
        continue

    # leaderboard CSV
    lb = make_leaderboard(df, pcols, suf)
    lb.to_csv(STAGE_DIR/f"Stage7_Rank_{flav}_{SWAN_YEAR}.csv", index=False)

    # plots
    hist_grid(df, pcols,
              f"{info['title']} – distribution by metric",
              STAGE_DIR/f"Stage7_HistGrid_{flav}_{SWAN_YEAR}.png")
    box_plot(df, pcols,
             f"{info['title']} – box-plot",
             STAGE_DIR/f"Stage7_Boxplot_{flav}_{SWAN_YEAR}.png")

    flavour_lb[flav] = lb
    log.info("✓ outputs for %s flavour written", flav)

# ──────────────── comparison: speed vs depth (if both) ────────────
if {"speed", "depth"} <= flavour_lb.keys():
    comp = (flavour_lb["speed"]
              .merge(flavour_lb["depth"], on="Metric",
                     suffixes=("_Speed", "_Depth"))
              .assign(Diff=lambda d: d["MeanProb_Speed"] - d["MeanProb_Depth"])
              .sort_values("MeanProb_Speed", ascending=False))
    comp.to_csv(STAGE_DIR/f"Stage7_Speed_vs_Depth_{SWAN_YEAR}.csv", index=False)

    plt.figure(figsize=(10, 4))
    sns.barplot(data=comp, x="Metric", y="Diff", palette="vlag")
    plt.title("Speed minus Depth (mean probability)")
    plt.ylabel("Δ (Speed – Depth)")
    plt.axhline(0, color="k")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(STAGE_DIR/f"Stage7_SpeedDepth_DiffBar_{SWAN_YEAR}.png", dpi=110)
    plt.close()
    log.info("✓ speed-vs-depth comparison written")

log.info("🎉  Stage 07 complete – artefacts in %s", STAGE_DIR)


